### Jennifer Albrecht - Predictive Analytics Week 11 - DASH
#### Resources:
    -Hands-On Predictive Analytics with Python: Chapter 9 – Implementing a Model with Dash 

### Import anticipated libraries

In [17]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import os
from dash.dependencies import Input, Output
from keras.models import load_model
import sklearn.externals
import joblib

from keras.models import Sequential
from keras.layers import Dense

### Setup and Import anticipated dataset

In [18]:
DATA_DIR='C:/Users/smurf/Downloads/'
FILE_NAME='travel_insurance.csv'
data_path=os.path.join(DATA_DIR,FILE_NAME)
df=pd.read_csv(data_path)


In [19]:
df=df.rename(columns={'Commision (in value)':'Commission','Net Sales':'NetSales','Agency Type':'AgencyType',
                      'Distribution Channel':'Distribution','Product Name':'ProductName'})
df.NetSales.astype(int)
df.Commission.astype(int)
df.Age.astype(int)
df.Duration.astype(int)

0        186
1        186
2         65
3         60
4         79
        ... 
63321    111
63322     58
63323      2
63324      3
63325     22
Name: Duration, Length: 63326, dtype: int32

### Prepare dataset

In [20]:
df['Duration']=df['Duration'].replace({-2:0,-1:0})
duration=df.loc[(df['Duration']>0) | (df['Commission']>0)]
df.loc[63326,'Duration']=df['Duration'].median()
df.loc[63326,'NetSales']=df['NetSales'].median()
df.loc[63326,'Commission']=df['Commission'].median()
df.loc[63326,'Age']=df['Age'].median()
df=df.loc[~((df['Commission'] > 118 )| (df['NetSales'] >118 ))]
df=pd.concat([df,pd.get_dummies(df['Agency'],prefix='Agency',drop_first=True)],axis=1)
df=pd.concat([df,pd.get_dummies(df['AgencyType'],prefix='AT',drop_first=True)],axis=1)
df=pd.concat([df,pd.get_dummies(df['Distribution'],prefix='DC',drop_first=True)],axis=1)
df=pd.concat([df,pd.get_dummies(df['ProductName'],prefix='PN',drop_first=True)],axis=1)
df=pd.concat([df,pd.get_dummies(df['Claim'],prefix='Claim',drop_first=True)],axis=1)
df=pd.concat([df,pd.get_dummies(df['Destination'],prefix='Dest',drop_first=True)],axis=1)

df

,Agency,AgencyType,Distribution,ProductName,Claim,Duration,Destination,NetSales,Commission,Gender,...,Dest_UNITED KINGDOM,Dest_UNITED STATES,Dest_URUGUAY,Dest_UZBEKISTAN,Dest_VANUATU,Dest_VENEZUELA,Dest_VIET NAM,"Dest_VIRGIN ISLANDS, U.S.",Dest_ZAMBIA,Dest_ZIMBABWE
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,-29.00,9.57,F,...,0,0,0,0,0,0,0,0,0,0
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,-29.00,9.57,F,...,0,0,0,0,0,0,0,0,0,0
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,-49.50,29.70,NaN,...,0,0,0,0,0,0,0,0,0,0
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,-39.60,23.76,NaN,...,0,0,0,0,0,0,0,0,0,0
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,-19.80,11.88,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63322,JZI,Airlines,Online,Basic Plan,No,58.0,CHINA,40.00,14.00,F,...,0,0,0,0,0,0,0,0,0,0
63323,JZI,Airlines,Online,Basic Plan,No,2.0,MALAYSIA,18.00,6.30,M,...,0,0,0,0,0,0,0,0,0,0
63324,JZI,Airlines,Online,Basic Plan,No,3.0,VIET NAM,18.00,6.30,M,...,0,0,0,0,0,0,1,0,0,0
63325,JZI,Airlines,Online,Basic Plan,No,22.0,HONG KONG,26.00,9.10,F,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df.describe()

,Duration,NetSales,Commission,Age,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,Agency_CWT,...,Dest_UNITED KINGDOM,Dest_UNITED STATES,Dest_URUGUAY,Dest_UZBEKISTAN,Dest_VANUATU,Dest_VENEZUELA,Dest_VIET NAM,"Dest_VIRGIN ISLANDS, U.S.",Dest_ZAMBIA,Dest_ZIMBABWE
count,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,...,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000,60495.000000
mean,39.089875,32.564033,6.901999,39.856335,0.005422,0.106786,0.001670,0.003207,0.001389,0.133813,...,0.019820,0.037210,0.000017,0.000165,0.000066,0.000083,0.027589,0.000017,0.000050,0.000050
std,86.414699,26.949696,12.197633,13.896785,0.073435,0.308843,0.040826,0.056539,0.037238,0.340454,...,0.139382,0.189277,0.004066,0.012856,0.008131,0.009091,0.163794,0.004066,0.007042,0.007042
min,0.000000,-389.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,18.000000,0.000000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,21.000000,26.000000,0.000000,36.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,48.000000,42.000000,9.250000,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4881.000000,118.000000,112.860000,118.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
df.columns

Index(['Agency', 'AgencyType', 'Distribution', 'ProductName', 'Claim',
       'Duration', 'Destination', 'NetSales', 'Commission', 'Gender',
       ...
       'Dest_UNITED KINGDOM', 'Dest_UNITED STATES', 'Dest_URUGUAY',
       'Dest_UZBEKISTAN', 'Dest_VANUATU', 'Dest_VENEZUELA', 'Dest_VIET NAM',
       'Dest_VIRGIN ISLANDS, U.S.', 'Dest_ZAMBIA', 'Dest_ZIMBABWE'],
      dtype='object', length=200)

In [43]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1, random_state=123)
df['dim_index']=pca.fit_transform(df[['Duration','Age','NetSales']])
#df.drop(['Duration','Net Sales'], axis=1,inplace=True)
X=df.drop(['Agency','AgencyType','Distribution','Claim','ProductName','Destination','Gender'],axis = 1)
y=np.log(df['Commission'])
numerical_features =['Duration','NetSales','Age']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X.loc[:, numerical_features] = scaler.fit_transform(X[numerical_features])

n_input = X.shape[1]
n_hidden1 = 32
n_hidden2 = 16
n_hidden3 = 8

nn_reg = Sequential()
nn_reg.add(Dense(units=n_hidden1, activation='relu', input_shape=(n_input,)))
nn_reg.add(Dense(units=n_hidden2, activation='relu'))
nn_reg.add(Dense(units=n_hidden3, activation='relu'))
# output layer
nn_reg.add(Dense(units=1, activation=None))

## Training the neural network
batch_size = 32
n_epochs = 40
nn_reg.compile(loss='mean_absolute_error', optimizer='adam')
nn_reg.fit(X, y, epochs=n_epochs, batch_size=batch_size)

C:\Users\smurf\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log



Epoch 1/40
1891/1891 [==============================] - 4s 2ms/step - loss: inf
Epoch 2/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 3/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 4/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 5/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 6/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 7/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 8/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 9/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 10/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 11/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 12/40
1891/1891 [==============================] - 3s 2ms/step - loss: inf
Epoch 13/40
1891/1891 [==============

In [44]:
## Serializing:
# PCA
joblib.dump(pca, 'C:/Users/smurf/Downloads/pca.joblib') 

# Scaler
joblib.dump(scaler, 'C:/Users/smurf/Downloads/scaler.joblib')

# Trained model
nn_reg.save("C:/Users/smurf/Downloads/travel.h5")

In [45]:
# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from keras.models import load_model


import numpy as np
import pandas as pd

app = dash.Dash(__name__)
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

model = load_model('C:/Users/smurf/Downloads/travel.h5')
pca = joblib.load('C:/Users/smurf/Downloads/pca.joblib')
scaler = joblib.load('C:/Users/smurf/Downloads/scaler.joblib')
model.predict(X)


1891/1891 [==============================] - 2s 1ms/step


array([[-2.2135020e+11],
       [-2.2679862e+11],
       [ 7.4937500e+04],
       ...,
       [-1.8515863e+11],
       [-5.3723535e+09],
       [-1.5460814e+11]], dtype=float32)

In [46]:
# Net Sales
input_netsales=dcc.Input(id='NetSales',value=32)
div_netsales=html.Div(
        children=[html.H3('NetSales'), input_netsales],
        className="six columns"
        )

# Duration
input_duration=dcc.Input(id='Duration',value=39)
div_duration=html.Div(
        children=[html.H3('Duration'), input_duration],
        className="six columns"
        )
# Age
input_age=dcc.Input(id='Age',value=36)
div_age=html.Div(
        children=[html.H3('Age'), input_age],
        className="six columns"
        )
# Commission
input_commission=dcc.Input(id='Commission',value=6)
div_commission=html.Div(
        children=[html.H3('Commission'), input_commission],
        className="six columns"
        )





In [47]:
#agency
agency_values = ['CBH' ,'CWT', 'JZI', 'KML', 'EPX', 'C2B', 'JWT' ,'RAB' ,'SSI' ,'ART' ,'CSR' ,'CCR', 'ADM' ,
                 'LWC' ,'TTW' ,'TST']
agency_options = [{'label': x, 'value': x} for x in agency_values]
input_agency = dcc.Dropdown(
    id='Agency',
    options = agency_options,
    value = 'RAB'
    )

div_agency = html.Div(
        children=[html.H3('Agency:'), input_agency],
        className="six columns"
        )

#agency type
at_values = ['Travel Agency', 'Airlines']
at_options = [{'label': x, 'value': x} for x in at_values]
input_at = dcc.Dropdown(
    id='AgencyType',        
    options = at_options,
    value = 'Travel Agency'
    )

div_at = html.Div(
        children=[html.H3('AgencyType:'), input_at],
        className="six columns"
        )

#Distribution channel
dc_values = ['Offline', 'Online']
dc_options = [{'label': x, 'value': x} for x in dc_values]
input_dc = dcc.Dropdown(
    id='Distribution',        
    options = dc_options,
    value = 'Offline'
    )

div_dc = html.Div(
        children=[html.H3('Distribution:'), input_dc],
        className="six columns"
        )


# product name
pn_values = ['Comprehensive Plan', 'Rental Vehicle Excess Insurance' ,'Value Plan', 'Basic Plan', 'Premier Plan' ,
             '2 way Comprehensive Plan' ,'Bronze Plan', 'Silver Plan' ,'Annual Silver Plan', 'Cancellation Plan',
             '1 way Comprehensive Plan' ,'Ticket Protector' ,'24 Protect' ,'Gold Plan', 'Annual Gold Plan' ,
             'Single Trip Travel Protect Silver', 'Individual Comprehensive Plan', 'Spouse or Parents Comprehensive Plan',
             'Annual Travel Protect Silver', 'Single Trip Travel Protect Platinum', 'Annual Travel Protect Gold' ,
             'Single Trip Travel Protect Gold', 'Annual Travel Protect Platinum', 'Child Comprehensive Plan',
             'Travel Cruise Protect', 'Travel Cruise Protect Family']
pn_options = [{'label': x, 'value': x} for x in pn_values]
input_pn = dcc.Dropdown(
    id='ProductName',        
    options = pn_options,
    value = '24 Protect'
    )

div_pn = html.Div(
        children=[html.H3('ProductName:'), input_pn],
        className="six columns"
        )


#claim
claim_values = ['Yes', 'No']
claim_options = [{'label': x, 'value': x} for x in claim_values]
input_claim = dcc.Dropdown(
    id='Claim',        
    options = claim_options,
    value = 'Yes'
    )

div_claim = html.Div(
        children=[html.H3('Claim:'), input_claim],
        className="six columns"
        )



#destination

dest_values = ['MALAYSIA' ,'AUSTRALIA', 'ITALY', 'UNITED STATES', 'THAILAND', "KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF" ,
               'NORWAY' ,'VIET NAM', 'DENMARK', 'SINGAPORE', 'JAPAN', 'UNITED KINGDOM' ,'INDONESIA' ,'INDIA',
               'CHINA', 'FRANCE', 'TAIWAN, PROVINCE OF CHINA' ,'PHILIPPINES', 'MYANMAR', 'HONG KONG', 'KOREA, REPUBLIC OF',
               'UNITED ARAB EMIRATES', 'NAMIBIA', 'NEW ZEALAND', 'COSTA RICA', 'BRUNEI DARUSSALAM', 'POLAND' ,'SPAIN',
               'CZECH REPUBLIC', 'GERMANY', 'SRI LANKA' ,'CAMBODIA', 'AUSTRIA' ,'SOUTH AFRICA', 
               'TANZANIA, UNITED REPUBLIC OF', "LAO PEOPLE'S DEMOCRATIC REPUBLIC" ,'NEPAL', 'NETHERLANDS', 'MACAO',
               'CROATIA' ,'FINLAND', 'CANADA', 'TUNISIA', 'RUSSIAN FEDERATION', 'GREECE', 'BELGIUM', 'IRELAND',
               'SWITZERLAND' ,'CHILE', 'ISRAEL', 'BANGLADESH' ,'ICELAND', 'PORTUGAL', 'ROMANIA', 'KENYA' ,'GEORGIA',
               'TURKEY', 'SWEDEN', 'MALDIVES', 'ESTONIA' ,'SAUDI ARABIA', 'PAKISTAN', 'QATAR', 'PERU' ,'LUXEMBOURG',
               'MONGOLIA', 'ARGENTINA' ,'CYPRUS' ,'FIJI' ,'BARBADOS', 'TRINIDAD AND TOBAGO' ,'ETHIOPIA' ,
               'PAPUA NEW GUINEA', 'SERBIA', 'JORDAN', 'ECUADOR' ,'BENIN' ,'OMAN', 'BAHRAIN', 'UGANDA' ,'BRAZIL' ,'MEXICO',
               'HUNGARY','AZERBAIJAN', 'MOROCCO', 'URUGUAY', 'MAURITIUS' ,'JAMAICA' ,'KAZAKHSTAN', 'GHANA' ,'UZBEKISTAN' ,
               'SLOVENIA', 'KUWAIT', 'GUAM' ,'BULGARIA' ,'LITHUANIA', 'NEW CALEDONIA' ,'EGYPT' ,'ARMENIA' ,'BOLIVIA' ,
               'VIRGIN ISLANDS, U.S.', 'PANAMA', 'SIERRA LEONE' ,'COLOMBIA' ,'PUERTO RICO', 'UKRAINE', 'GUINEA', 
               'GUADELOUPE' ,'MOLDOVA, REPUBLIC OF' ,'GUYANA', 'LATVIA', 'ZIMBABWE', 'VANUATU' ,'VENEZUELA', 'BOTSWANA' ,
               'BERMUDA', 'MALI' ,'KYRGYZSTAN', 'CAYMAN ISLANDS', 'MALTA', 'LEBANON' ,'REUNION', 'SEYCHELLES', 'ZAMBIA',
               'SAMOA', 'NORTHERN MARIANA ISLANDS', 'NIGERIA' ,'DOMINICAN REPUBLIC', 'TAJIKISTAN', 'ALBANIA' ,
               'MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF', 'LIBYAN ARAB JAMAHIRIYA' ,'ANGOLA' ,'BELARUS',
               'TURKS AND CAICOS ISLANDS', 'FAROE ISLANDS', 'TURKMENISTAN', 'GUINEA-BISSAU' ,'CAMEROON', 'BHUTAN',
               'RWANDA' ,'SOLOMON ISLANDS', 'IRAN, ISLAMIC REPUBLIC OF' ,'GUATEMALA', 'FRENCH POLYNESIA' ,'TIBET' ,
               'SENEGAL', 'REPUBLIC OF MONTENEGRO', 'BOSNIA AND HERZEGOVINA']
dest_options = [{'label': x, 'value': x} for x in dest_values]
input_dest = dcc.Dropdown(
    id='Destination',        
    options = dest_options,
    value = 'PANAMA'
    )

div_dest = html.Div(
        children=[html.H3('Destination:'), input_dest],
        className="six columns"
        )


In [48]:
div_numerical = html.Div(
        children = [div_netsales, div_duration, div_age],
        className="row"
        )

In [49]:
div_dimensions = html.Div(
        children = [div_commission],
        className="row"
        )

In [50]:
div_categorical = html.Div(
        children = [div_agency, div_at, div_dc, div_pn, div_claim, div_dest],
        className="row"
        )

In [53]:
def get_prediction(Duration, NetSales, Age, Agency, AgencyType, Distribution, ProductName, Claim, Destination):
    cols = ['Agency', 'Agency Type', 'Distribution Channel', 'Product Name','Claim', 'Duration', 'Destination',
            'Net Sales', 'Commision (in value)','Gender',
            'Agency_ART','Agency_C2B','Agency_CBH','Agency_CCR','Agency_CSR','Agency_CWT', 'Agency_JZI' 'Agency_KML',
            'Agency_EPX', 'Agency_JWT', 'Agency_RAB', 'Agency_SSI', 'Agency_ADM' ,'Agency_LWC', 'Agency_TTW', 'Agency_TST',
            'AT_Travel Agency' ,'AT_Airlines',
            'DC_Offline', 'DC_Online',
            'PN_Comprehensive Plan', 'PN_Rental Vehicle Excess Insurance', 'PN_Value Plan', 'PN_Basic Plan',
            'PN_Premier Plan', 'PN_2 way Comprehensive Plan' ,'PN_Bronze Plan', 'PN_Silver Plan', 'PN_Annual Silver Plan',
            'PN_Cancellation Plan', 'PN_1 way Comprehensive Plan' ,'PN_Ticket Protector', 'PN_24 Protect', 'PN_Gold Plan',
            'PN_Annual Gold Plan', 'PN_Single Trip Travel Protect Silver', 'PN_Individual Comprehensive Plan',
            'PN_Spouse or Parents Comprehensive Plan', 'PN_Annual Travel Protect Silver', 
            'PN_Single Trip Travel Protect Platinum', 'PN_Annual Travel Protect Gold', 'PN_Single Trip Travel Protect Gold',
            'PN_Annual Travel Protect Platinum', 'PN_Child Comprehensive Plan', 'PN_Travel Cruise Protect', 
            'PN_Travel Cruise Protect Family',
            'Claim_Yes', 'Claim_No',
            'Dest_MALAYSIA', 'Dest_AUSTRALIA', 'Dest_ITALY', 'Dest_UNITED STATES' ,'Dest_THAILAND', 
            "Dest_KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF" ,'Dest_NORWAY', 'Dest_VIET NAM' ,'Dest_DENMARK', 'Dest_SINGAPORE',
            'Dest_JAPAN', 'Dest_UNITED KINGDOM', 'Dest_INDONESIA' ,'Dest_INDIA', 'Dest_CHINA', 'Dest_FRANCE', 
            'Dest_TAIWAN, PROVINCE OF CHINA', 'Dest_PHILIPPINES', 'Dest_MYANMAR' ,'Dest_HONG KONG', 
            'Dest_KOREA, REPUBLIC OF', 'Dest_UNITED ARAB EMIRATES' ,'Dest_NAMIBIA', 'Dest_NEW ZEALAND', 'Dest_COSTA RICA',
            'Dest_BRUNEI DARUSSALAM' ,'Dest_POLAND' ,'Dest_SPAIN', 'Dest_CZECH REPUBLIC', 'Dest_GERMANY', 'Dest_SRI LANKA',
            'Dest_CAMBODIA', 'Dest_AUSTRIA', 'Dest_SOUTH AFRICA', 'Dest_TANZANIA, UNITED REPUBLIC OF',
            "Dest_LAO PEOPLE'S DEMOC,RATIC REPUBLIC" ,'Dest_NEPAL','Dest_NETHERLANDS', 'Dest_MACAO', 'Dest_CROATIA',
            'Dest_FINLAND', 'Dest_CANADA', 'Dest_TUNISIA', 'Dest_RUSSIAN FEDERATION' ,'Dest_GREECE', 'Dest_BELGIUM',
            'Dest_IRELAND', 'Dest_SWITZERLAND', 'Dest_CHILE', 'Dest_ISRAEL', 'Dest_BANGLADESH', 'Dest_ICELAND',
            'Dest_PORTUGAL', 'Dest_ROMANIA', 'Dest_KENYA', 'Dest_GEORGIA', 'Dest_TURKEY', 'Dest_SWEDEN', 'Dest_MALDIVES',
            'Dest_ESTONIA', 'Dest_SAUDI ARABIA', 'Dest_PAKISTAN', 'Dest_QATAR', 'Dest_PERU', 'Dest_LUXEMBOURG', 
            'Dest_MONGOLIA', 'Dest_ARGENTINA' ,'Dest_CYPRUS', 'Dest_FIJI', 'Dest_BARBADOS', 'Dest_TRINIDAD AND TOBAGO',
            'Dest_ETHIOPIA', 'Dest_PAPUA NEW GUINEA', 'Dest_SERBIA', 'Dest_JORDAN', 'Dest_ECUADOR', 'Dest_BENIN', 
            'Dest_OMAN', 'Dest_BAHRAIN' ,'Dest_UGANDA' ,'Dest_BRAZIL', 'Dest_MEXICO', 'Dest_HUNGARY', 'Dest_AZERBAIJAN',
            'Dest_MOROCCO' ,'Dest_URUGUAY', 'Dest_MAURITIUS', 'Dest_JAMAICA', 'Dest_KAZAKHSTAN', 'Dest_GHANA',
            'Dest_UZBEKISTAN', 'Dest_SLOVENIA', 'Dest_KUWAIT', 'Dest_GUAM', 'Dest_BULGARIA', 'Dest_LITHUANIA', 
            'Dest_NEW CALEDONIA', 'Dest_EGYPT', 'Dest_ARMENIA', 'Dest_BOLIVIA', 'Dest_VIRGIN ISLANDS, U.S.', 'Dest_PANAMA',
            'Dest_SIERRA LEONE', 'Dest_COLOMBIA' ,'Dest_PUERTO RICO' ,'Dest_UKRAINE', 'Dest_GUINEA', 'Dest_GUADELOUPE',
            'Dest_MOLDOVA, REPUBLIC OF' ,'Dest_GUYANA' ,'Dest_LATVIA', 'Dest_ZIMBABWE', 'Dest_VANUATU', 'Dest_VENEZUELA',
            'Dest_BOTSWANA', 'Dest_BERMUDA' ,'Dest_MALI', 'Dest_KYRGYZSTAN', 'Dest_CAYMAN ISLANDS', 'Dest_MALTA', 
            'Dest_LEBANON', 'Dest_REUNION', 'Dest_SEYCHELLES', 'Dest_ZAMBIA', 'Dest_SAMOA' ,'Dest_NORTHERN MARIANA ISLANDS',
            'Dest_NIGERIA', 'Dest_DOMINICAN REPUBLIC', 'Dest_TAJIKISTAN', 'Dest_ALBANIA', 
            'Dest_MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF', 'Dest_LIBYAN ARAB JAMAHIRIYA', 'Dest_ANGOLA' ,
            'Dest_BELARUS', 'Dest_TURKS AND CAICOS ISLANDS', 'Dest_FAROE ISLANDS' ,'Dest_TURKMENISTAN', 'Dest_GUINEA-BISSAU',
            'Dest_CAMEROON', 'Dest_BHUTAN', 'Dest_RWANDA', 'Dest_SOLOMON ISLANDS', 'Dest_IRAN, ISLAMIC REPUBLIC OF',
            'Dest_GUATEMALA', 'Dest_FRENCH POLYNESIA', 'Dest_TIBET' ,'Dest_SENEGAL', 'Dest_REPUBLIC OF MONTENEGRO',
            'Dest_BOSNIA AND HERZEGOVINA', 'dim_index']

    agency_dict = {x: 'Agency_' + x for x in agency_values[1:]}
    at_dict = {x: 'AT_' + x for x in at_values[1:]}
    dc_dict = {x: 'DC_' + x for x in dc_values[1:]}
    pn_dict = {x: 'PN_' + x for x in pn_values[1:]}
    claim_dict = {x: 'Claim_' + x for x in claim_values[1:]}
    dest_dict = {x: 'Dest_' + x for x in dest_values[1:]}
    
    ## produce a dataframe with a single row of zeros
    df = pd.DataFrame(data = np.zeros((1,len(cols))), columns = cols)
    
    ## get the numeric characteristics
    df.loc[0,'NetSales'] = NetSales
    df.loc[0,'Age'] = Age
    df.loc[0,'Duration'] = Duration
    
    ## transform dimensions into a single dim_index using PCA
    dims_df = pd.DataFrame(data=[[df]], columns=['Commission'])
    df.loc[0,'dim_index'] = pca.transform(dims_df).flatten()[0]

    ## Use the one-hot encoding for the categorical features
    if Agency!='RAB':
        df.loc[0, agency_dict[Agency]] = 1
    
    if AT!='Travel Agency':
        df.loc[0, at_dict[AT]] = 1
    
    if DC != 'Offline':
        df.loc[0, dc_dict[DC]] = 1
    
    if PN != '24 Protect':
        df.loc[0, pn_dict[PN]] = 1
    
    if Claim != 'Yes':
        df.loc[0, claim_dict[Claim]] = 1
    
    if Dest != 'United States':
        df.loc[0, dest_dict[Dest]] = 1
    
    ## Scale the numerical features using the trained scaler
    numerical_features = ['NetSales', 'Age', 'Duration']
    df.loc[:,numerical_features] = scaler.transform(df.loc[:,numerical_features])
    
    ## Get the predictions using our trained neural network
    prediction = model.predict(df.values).flatten()[0]
    
    ## Transform the log-prices to prices
    prediction = np.exp(prediction)
   
    return int(prediction)
    


In [54]:
## App layout
app.layout = html.Div([
        html.H1('Predict Sales'),
        
        html.H2('Enter the travel characteristics to get the predicted price'),
        
        html.Div(
                children=[div_numerical, div_dimensions, div_categorical]
                ),
        html.H1(id='output',
                style={'margin-top': '50px', 'text-align': 'center'})
        ])

predictors = ['Agency', 'AgencyType', 'Distribution', 'ProductName', 'Claim', 'Duration', 'Destination', 'NetSales',
              'Commission']
@app.callback(
        Output('output', 'children'),
        [Input(x, 'value') for x in predictors],allow_duplicate=True)
def show_prediction(Duration, NetSales, Age, Agency, AgencyType, Distribution, ProductName, Claim, Destination): 
    pred = get_prediction(Duration, NetSales, Age, Agency, AgencyType, Distribution, ProductName, Claim, Destination)
    return str("Predicted Price: {:,}".format(pred))


if __name__ == '__main__':
    app.run_server(debug=True)

C:\Users\smurf\anaconda3\lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



### Utilize information from above